<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Background

Before we move into the exercises, we'll cover some background information to get you started.

In [12]:
import numpy as np
import pandas as pd
import requests
import re

## Markov Chain Simple Example

Markov chains are a way of representing how systems change over time. The main concept behind Markov chains are that they are memoryless, meaning that the next state of a process only depends on the previous state.

![image](https://upload.wikimedia.org/wikipedia/commons/7/7a/Markov_Chain_weather_model_matrix_as_a_graph.png)

The way to read the Markov chain above from [Wikipedia](https://commons.wikimedia.org/w/index.php?curid=25300524) is:
* If I am currently in the sunny state, there is a 10% chance I will go to the rainy state and a 90% chance I will remain in the sunny state
* If I am currently in the rainy state, there is an 50% chance I will go to the sunny state and a 50% chance I will remain in the rainy state

## Transition Matrices

This is what our **transition matrix** will look like for the Markov chain diagram above. Take a minute to interpret the rows and columns of this matrix.

In [13]:
P = np.asarray([.9, .1, .5, .5]).reshape(2,2)
states = ['sunny', 'rainy']

pd.DataFrame(P, index=states, columns=states)

,sunny,rainy
sunny,0.9,0.1
rainy,0.5,0.5


## Predict Tomorrow's Weather

Let's say it's sunny today, we can represent that as:

`today = [1, 0]`

**Predict tomorrow's weather using what you know about today and the transition matrix.**

In [14]:
today = [1, 0]

tomorrow = np.dot(today, P)
tomorrow

array([0.9, 0.1])

In this example, there is a 90% chance it will remain sunny tomorrow, and a 10% chance it'll be rainy.

**Predict the day after tomorrow's weather.**

In [15]:
# Method 1: Multiply tomorrow's weather by the transition matrix
day_after = np.dot(tomorrow, P)
day_after

array([0.86, 0.14])

In [16]:
# Method 2: Multiply today's weather by the transition matrix^2
day_after = np.dot(today, np.linalg.matrix_power(P, 2))
day_after

array([0.86, 0.14])

## Text Generation

Markov chains can also be used for very basic text generation. **Think about every word in a corpus as a state.** We can make a simple assumption that the next word is only dependent on the previous word - which is the basic assumption of a Markov chain. In this exercise, you'll create a text generator which uses only this concept.

## Read in some text to imitate

We are going to generate some text in the style of inspirational quotes, so let's first read in the data.

In [17]:
url = 'https://raw.githubusercontent.com/leontoddjohnson/datasets/main/text/inspiration_quotes.txt'

content = requests.get(url)
quotes_raw = content.text

print(quotes_raw[:1000])

“Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.” —Peter Shepherd

“Life is a journey and if you fall in love with the journey you will be in love forever.” —Peter Hagerty

“When you return to your old hometown, you find it wasn’t the town you missed, but your childhood.” —Earl Wilson

“As we grow old, the beauty steals inward.” —Ralph Waldo Emerson

“Life begins as a quest of the child for the man, and ends as a journey by the man to rediscover the child.” —Sam Ewing

Happiness
“Ultimately your greatest teacher is to live with an open heart.” —Emmanuel (Pat Rodegast)

“Doing what you like is freedom. Liking what you do is happiness.” —Frank Tyger

“We forge the chains we wear in life.” —Charles Dickens

happiness quote
“If you look to others for fulfillment, you will never be fulfilled. If your happiness depends on money, you will never be happy with yourself. Be content with what you 

## Clean up the text data

There are many ways to clean up data before building a text generator. In this case, we'll try to at least just extract the quotes themselves.

*After you complete the exercises, feel free to adjust this section of the process ...*

In [18]:
quotes = quotes_raw.replace('\n', ' ')
quotes = re.split("[“”]", quotes)   # split on the unique “ characters
quotes[:3]

['',
 'Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.',
 ' —Peter Shepherd  ']

In [19]:
# skip the first one, and capture every other element
quotes = quotes[1::2]
quotes[:3]

['Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions.',
 'Life is a journey and if you fall in love with the journey you will be in love forever.',
 'When you return to your old hometown, you find it wasn’t the town you missed, but your childhood.']

In [20]:
# create one long corpus of text
corpus = ' '.join(quotes)

# remove long whitespaces (see regex101.com)
corpus = re.sub(r"\s+", " ", corpus)

# remove leading/trailing whitespaces
corpus = corpus.strip()

corpus[:200]

'Healing comes from taking responsibility: to realize that it is you - and no one else - that creates your thoughts, your feelings, and your actions. Life is a journey and if you fall in love with the '

In general, this version of `corpus` should work just fine!

# Exercises

In these exercises, you'll build a `MarkovText` object that takes in a `corpus` of text, and has the capability to generate text using the Markov Property.

In [21]:
from apputil import *

## Exercise 1: Build a Transition Dictionary

Update the `get_term_dict` method to build a term dictionary of Markov states with the following traits:

* The keys should be (unique) tokens in the corpus
* For each key, the value should be a `list` of all the tokens that follow that key
    - E.g., if my total corpus is "Astrid is very kind, is she not?", then my dictionary should include `{... "is": ["very", "she"] ...}`.
    - Decide whether or not to include duplicates (i.e., *every* iteration) in these lists. Then, explain why or why not.

*Hint: You'll likely want to use [`defaultdict(list)`](https://realpython.com/python-defaultdict/#understanding-the-python-defaultdict-type) here.*

Apply the function to the quotes above. Your final output should look something like this:
    
```python
{
    'Healing': ['comes'],
    'comes': ['from', 'the', 'the', ...],
    'from': ['taking', 'aesthetic', 'a'],
    ...
}
```

In [22]:
text_gen = MarkovText(corpus, k=1)

text_gen.get_term_dict()

{'Healing': ['comes'],
 'comes': ['from', 'from'],
 'from': ['taking',
  'aesthetic',
  'a',
  'having',
  'not',
  'the',
  'the',
  'it.',
  'our',
  'experience',
  'experience.',
  'avoiding',
  'our'],
 'taking': ['responsibility:', 'it.'],
 'responsibility:': ['to'],
 'to': ['realize',
  'your',
  'rediscover',
  'live',
  'others',
  'you.',
  'be',
  'be',
  'pay',
  'give',
  'share',
  'have',
  'draw',
  'think',
  'you;',
  'go',
  'the',
  'obtain',
  'mourn',
  'live',
  'a',
  'make',
  'forgive.',
  'take',
  'do.',
  'be',
  'be',
  'change',
  'control',
  'an',
  'take',
  'accept',
  'change',
  'know',
  'win,',
  'be',
  'succeed,',
  'live',
  'the',
  'order,',
  'clarity.',
  'get',
  'be',
  'shine',
  'manifest',
  'do',
  'work',
  'work',
  'think',
  'play',
  'read',
  'be',
  'happiness',
  'love',
  'share',
  'be',
  'laugh',
  'dream',
  'a',
  'make',
  'others;',
  'prove',
  'learn',
  'focus',
  'anything,',
  'accept',
  'change',
  'know',
  'tr

## Exercise 2: Create a text generator

Update the `generate()` method to generate sentences using the Markov property.

- This should use the `.term_dict`, and it should take in the number of terms you want generated, `term_count`.
- Your function should also be able to accept an *optional* user-defined `seed_term` that starts the generator. By default, you can have the function select a random first token.
  - If the user-defined seed term is not in the corpus, raise a `ValueError`.
- Each "next word" should be chosen *at random* given the "current" word and the term dictionary.
  - Consider [`numpy.random.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) for this.

*Hint: think about what happens if the last term in the corpus occurs only **once**.*

In [23]:
text_gen.generate()

'confidence, a pebble in life is beauty, admire it. Life is unpopular is measured by'

*Note: this exercise illustrates both a Markov Chain (with constant transition probabilities) **and** a [Monte Carlo Simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method) (iterative sampling from a constantly defined probability distribution of words)!*

---

# Bonus Exercise

*<font color='darkorange'>The following exercise is completely optional</font>, and will not be reviewed unless explicitly requested. That said, these are interesting to work on if you have the time!*

Adjust your text generator such that we can increase the "state window size" from one word to `k` words. That is, the term dictionary in this version with `k = 2` would look something like this (*note the tuple keys*):

```python
{
    ("Healing", "comes"): ["from", ...],
    ...,
    ("it", "is"): ["you", "very"],
    ...
}
```

Generate some text for `k` values of 1, 2, and 3, and compare the differences in text coherence. Try larger corpuses ...

In [24]:
text_gen = MarkovText(corpus, k=2)

text_gen.get_term_dict()

{('Healing', 'comes'): ['from'],
 ('comes', 'from'): ['taking', 'not'],
 ('from', 'taking'): ['responsibility:'],
 ('taking', 'responsibility:'): ['to'],
 ('responsibility:', 'to'): ['realize'],
 ('to', 'realize'): ['that'],
 ('realize', 'that'): ['it'],
 ('that', 'it'): ['is', 'is'],
 ('it', 'is'): ['you',
  'sufficient',
  'in',
  'the',
  'the',
  'the',
  'the',
  'the',
  'the',
  'too',
  'the',
  'hitching',
  'unpopular',
  'not'],
 ('is', 'you'): ['-'],
 ('you', '-'): ['and'],
 ('-', 'and'): ['no'],
 ('and', 'no'): ['one'],
 ('no', 'one'): ['else'],
 ('one', 'else'): ['-'],
 ('else', '-'): ['that'],
 ('-', 'that'): ['creates'],
 ('that', 'creates'): ['your'],
 ('creates', 'your'): ['thoughts,'],
 ('your', 'thoughts,'): ['your', 'liberates'],
 ('thoughts,', 'your'): ['feelings,'],
 ('your', 'feelings,'): ['and'],
 ('feelings,', 'and'): ['your'],
 ('and', 'your'): ['actions.'],
 ('your', 'actions.'): ['Life'],
 ('actions.', 'Life'): ['is'],
 ('Life', 'is'): ['a',
  'an',
  'beau

In [25]:
text_gen = MarkovText(corpus, k=3)

text_gen.get_term_dict()

{('Healing', 'comes', 'from'): ['taking'],
 ('comes', 'from', 'taking'): ['responsibility:'],
 ('from', 'taking', 'responsibility:'): ['to'],
 ('taking', 'responsibility:', 'to'): ['realize'],
 ('responsibility:', 'to', 'realize'): ['that'],
 ('to', 'realize', 'that'): ['it'],
 ('realize', 'that', 'it'): ['is'],
 ('that', 'it', 'is'): ['you', 'not'],
 ('it', 'is', 'you'): ['-'],
 ('is', 'you', '-'): ['and'],
 ('you', '-', 'and'): ['no'],
 ('-', 'and', 'no'): ['one'],
 ('and', 'no', 'one'): ['else'],
 ('no', 'one', 'else'): ['-'],
 ('one', 'else', '-'): ['that'],
 ('else', '-', 'that'): ['creates'],
 ('-', 'that', 'creates'): ['your'],
 ('that', 'creates', 'your'): ['thoughts,'],
 ('creates', 'your', 'thoughts,'): ['your'],
 ('your', 'thoughts,', 'your'): ['feelings,'],
 ('thoughts,', 'your', 'feelings,'): ['and'],
 ('your', 'feelings,', 'and'): ['your'],
 ('feelings,', 'and', 'your'): ['actions.'],
 ('and', 'your', 'actions.'): ['Life'],
 ('your', 'actions.', 'Life'): ['is'],
 ('action